In [ ]:
"""
Unabashedly following Jake van der Plas' excellent overview paper:
https://iopscience.iop.org/article/10.3847/1538-4365/aab766

Lecture 1 Outline:

1. Motivation: what is real data like? nonuniform, different sized error bars.
   Why LS? It's the most common, and its generally fast. Major classes of periodicity 
   detection: phase folding, fourier-based, least-squares model-fitting 
2a. review: PSD
2b. review: convolution thm
3. Effect of (uniform) sampling (3.2, fig 7)
4. Effect of nonuniform sampling (4, figs 9 & 10)
5. Effect of semistructured observing windows (4.2, figs 14 & 15)
6. Definitions: beating, aliasing, window function

Problem ideas:
- given a window function and a true signal, draw the LS periodogram
- identify beat frequencies and alias frequencies in LS of real data
- decide a follow-up strategy based on a window function & a LS periodogram
- (more)

Lecture 2 Outline:

1. How LS relates to classical periodogram relates to PSD (they are estimators of PSD)
2. Sketch derivations of LS: smart choice of coefficients (Scargle) + least squares (Lomb)
3. Some useful extensions: uncertainties (6.1), floating mean (6.2), higher-order modes (6.3)
4. Computing the window function & the CLEAN algorithm
5. FAPS and eFAPS
6. When the LS periodogram fails: ex: P/2 being isolated in rotation curve. 
7. When the LS periodogram fails: correlated noise

Problem ideas:
- compute the actual periodogram and compare it to the LS periodogram
- use astropy periodogram, investigate normalization schemes
- numerically compute window function, eFAP, and LS (incorporating/not 
  uncertainties & floating mean & higher-order modes)
- (more)

"""

# Introduction to Time Series: Lomb Scargle Periodograms I

Sarah Blunt, CIERA/Northwestern
Presented at LSST Discovery Alliance Data Science Fellowship Program Session 20: Time Series

TODO: adequately cite van der Plas paper in a way that communicates I've used a lot of ideas and code from his paper. 

#### Goals for this Lesson:
TODO: fill in

### I: Motivation
(5 mins)

Real data are messy. Let's take a look at an example of a real dataset we might want to analyze and identify some problems we might run into constucting a power spectrum.

![data-example](data-and-images/vanderPlas-fig1.jpg)

What are some problems you notice?
- Discrete samples (i.e. not continuous)
- Non-uniform sampling
- Error bars exist
- The error bars are different sizes
- There are big gaps in the data
- The gaps themselves are somewhat periodic (day/night, and also when the object is visible in the sky)

The Lomb-Scargle (LS) periodogram provides a framework for analyzing datasets like this one. However, it's not a perfect tool! The more you understand about how it works, the less likely you will be to misinterpret a LS periodogram. 

Why are we choosing to focus on the Lomb-Scargle periodogram? There are many, many other methods for detecting periodicity in "real" data, which broadly fall into the following categories:
- Fourier methods, which are derived from the Fourier transform
- Phase-folding methods
- Least-squares methods, and
- Bayesian methods

These are not really separate categories; for example, one can construct a probability function using a phase-folding or least-squares method, then incorporate it into a Bayesian analysis. There are two main reasons that we are devoting so much time to the LS periodogram:

Reason #1: It's super popular! You are very likely to encounter the LS periogoram in time-series analysis, so it's important to have a solid understanding.

Reason #2: It is related to all four "classes" of methods above. It can be derived from Fourier theory (which we will show), and can also be viewed as a least-squares method (which we will also show). We won't discuss this, but it can also be derived from Bayesian probability theory and is closely related to phase-folding techniques. Therefore, gaining intuition about the LS periodogram will help you understand many of these methods, and give you a baseline for understanding other techniques.

### II: Brief Review of the Continuous Fourier Transform
(10 mins)

We will derive the LS periodogram from Fourier analysis, so let's briefly restate some necessary background.

#### The Power Spectral Density (PSD)

The Fourier transform of a periodic cosine function is:

$$
\mathcal{F}\left\{\cos \left(2 \pi f_0 t\right)\right\}=\frac{1}{2}\left[\delta\left(f-f_0\right)+\delta\left(f+f_0\right)\right]
$$

In words, the Fourier transform of a periodic function is the sum of two delta functions: one centered at $-f_0$ and one at $+f_0$. The Fourier transform of a periodic sine function, on the other hand, is:

$$
\mathcal{F}\left\{\sin \left(2 \pi f_0 t\right)\right\}=\frac{1}{2i}\left[\delta\left(f-f_0\right)-\delta\left(f+f_0\right)\right],
$$

which is actually just $\mathcal{F}\left\{\cos \left(2 \pi f_0 t\right)\right\}$ multiplied by $e^{-2\pi i f (\pi/2)}$. Notice that $\mathcal{F}\left\{\cos \left(2 \pi f_0 t\right)\right\}$ is purely real, and $\mathcal{F}\left\{\sin \left(2 \pi f_0 t\right)\right\}$ is purely imaginary. 

This motivates the definition of the power spectral density (PSD), which is the **squared magnitude of the Fourier transform**:

$$
\mathcal{P}_g \equiv|\mathcal{F}\{g\}|^2
$$

While the Fourier transform is a complex number (part real and part imaginary), the PSD is a *real, positive number*. It retains useful properties of the Fourier transform, while being insensitive to time shifts. In other words, while $\mathcal{F}\left\{\cos \left(2 \pi f_0 t\right)\right\}$ and $\mathcal{F}\left\{\sin \left(2 \pi f_0 t\right)\right\}$ have different Fourier transforms, they have the same PSD.

Problems:

1. Derive the Fourier transform of a sinusoidal function with a time shift. Compute its PSD, and show that it does not depend on the phase shift. This is a general result; the PSD of a time-shifted function will not affect the PSD. Write a sentence about why this is important for time series analysis.